In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd /content/gdrive/My Drive/ANN
!ls

/content/gdrive/My Drive/ANN
X_test.pickle  X_train.pickle  Y_test.pickle  Y_train.pickle


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import pickle
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [0]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [0]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [0]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

In [0]:
print(X_train.shape)
print(X_test.shape)

(1006, 150, 150, 3)
(607, 150, 150, 3)


In [0]:
dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
      NAME = "{}-nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
      print(NAME)

32-nodes-0-dense-1585650488
64-nodes-0-dense-1585650488
128-nodes-0-dense-1585650488
32-nodes-1-dense-1585650488
64-nodes-1-dense-1585650488
128-nodes-1-dense-1585650488
32-nodes-2-dense-1585650488
64-nodes-2-dense-1585650488
128-nodes-2-dense-1585650488


In [0]:
#%load_ext tensorboard
%tensorboard --logdir=logs

In [0]:
%reload_ext tensorboard

In [0]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

notebook.display(port=6006, height=1000)

Known TensorBoard instances:
  - port 6007: logdir logs (started 0:00:08 ago; pid 488)
  - port 6006: logdir logs (started 0:05:33 ago; pid 350)
Selecting TensorBoard with logdir logs (started 0:05:33 ago; port 6006, pid 350).


In [0]:
dense_layers = [0,1, 2]
layer_sizes = [32, 64, 128]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
      #   
      NAME = "nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
      print(NAME)
      model = Sequential()
      model.add(Flatten(input_shape=X_train.shape[1:]))
      for _ in range(dense_layer):
          model.add(Dense(layer_size))
          model.add(Activation('relu'))
          model.add(Dropout(0.33))

      model.add(Dense(4))
      model.add(Activation('softmax'))


      model.compile(loss='sparse_categorical_crossentropy',
        optimizer= "adam",
        metrics=['accuracy'],
        )
      
      #Fit the model
      model.fit(X_train, Y_train,
      batch_size=32,
      epochs=20,
      validation_data=(X_test,Y_test),
      callbacks=[tensorboard,es])
              
      #Score 
      scores = model.evaluate(X_test, Y_test, verbose=1) 
      print('Test loss:', scores[0])
      print('Test accuracy:', scores[1])
      
      #Save model
      model.save("{}-model-{}-accuracy.h5".format(NAME,scores[1]))
      
      #precision    recall  f1-score   support
      y_pred = model.predict(X_test, batch_size=64, verbose=1)
      y_pred_bool = np.argmax(y_pred, axis=1)
      print(classification_report(Y_test, y_pred_bool))
      print()
      print()  

nodes-32-dense-0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 2s 2ms/sample - loss: 922.3333 - acc: 0.4016 - val_loss: 456.9620 - val_acc: 0.5568
Epoch 2/20
1006/1006 [==============================] - 0s 254us/sample - loss: 229.3181 - acc: 0.6660 - val_loss: 156.5977 - val_acc: 0.6870
Epoch 3/20
1006/1006 [==============================] - 0s 246us/sample - loss: 126.3268 - acc: 0.7286 - val_loss: 159.4041 - val_acc: 0.6507
Epoch 4/20
1006/1006 [==============================] - 0s 274us/sample - loss: 79.7549 - acc: 0.7883 - val_loss: 138.1594 - val_acc: 0.7381
Epoch 5/20
1006/1006 [==============================] - 0s 251us/sample - loss: 103.1610 - acc: 0.7823 - val_loss: 138.1049 - val_acc: 0.7710
Epoch 6/20
1006/1006 [==============================] - 0s 255us/sample - loss: 53.0258 - acc: 0.8539 - val_loss: 122.6456 - val_acc: 0.7150
Epoch 7

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
1006/1006 [==============================] - 0s 430us/sample - loss: 448.6943 - acc: 0.2604 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 2/20
1006/1006 [==============================] - 0s 289us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 3/20
1006/1006 [==============================] - 0s 292us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 4/20
1006/1006 [==============================] - 0s 287us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 5/20
1006/1006 [==============================] - 0s 284us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 6/20
1006/1006 [==============================] - 0s 317us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3864 - val_acc: 0.2471
Epoch 00006: early stopping
607/607 [==============================] - 0s 153us/sample - loss: 1.3864 - acc: 0.2471
Test loss: 1.3863594490770845
Test accuracy: 0.2471169

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 0s 454us/sample - loss: 2215.2967 - acc: 0.3489 - val_loss: 129.8312 - val_acc: 0.4185
Epoch 2/20
1006/1006 [==============================] - 0s 330us/sample - loss: 19.6411 - acc: 0.3330 - val_loss: 1.3851 - val_acc: 0.2471
Epoch 3/20
1006/1006 [==============================] - 0s 317us/sample - loss: 1.3850 - acc: 0.2555 - val_loss: 1.3850 - val_acc: 0.2471
Epoch 4/20
1006/1006 [==============================] - 0s 311us/sample - loss: 1.3850 - acc: 0.2555 - val_loss: 1.3849 - val_acc: 0.2471
Epoch 5/20
1006/1006 [==============================] - 0s 348us/sample - loss: 1.3849 - acc: 0.2555 - val_loss: 1.3848 - val_acc: 0.2471
Epoch 6/20
1006/1006 [==============================] - 0s 350us/sample - loss: 1.3846 - acc: 0.2555 - val_loss: 1.3846 - val_acc: 0.2471
Epoch 7/20
1006/1006 [==============================] - 0s 334us/sample - loss: 1.3837 - acc: 0.2565 - val_loss: 1.3844

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 0s 494us/sample - loss: 338.8123 - acc: 0.2584 - val_loss: 1.3845 - val_acc: 0.2488
Epoch 2/20
1006/1006 [==============================] - 0s 301us/sample - loss: 1.3852 - acc: 0.2604 - val_loss: 1.3843 - val_acc: 0.2488
Epoch 3/20
1006/1006 [==============================] - 0s 341us/sample - loss: 1.3864 - acc: 0.2465 - val_loss: 1.3844 - val_acc: 0.2488
Epoch 4/20
1006/1006 [==============================] - 0s 335us/sample - loss: 1.3865 - acc: 0.2584 - val_loss: 1.3843 - val_acc: 0.2488
Epoch 5/20
1006/1006 [==============================] - 0s 342us/sample - loss: 1.3864 - acc: 0.2565 - val_loss: 1.3842 - val_acc: 0.2488
Epoch 6/20
1006/1006 [==============================] - 0s 354us/sample - loss: 1.3849 - acc: 0.2555 - val_loss: 1.3843 - val_acc: 0.2488
Epoch 7/20
1006/1006 [==============================] - 0s 403us/sample - loss: 1.3850 - acc: 0.2555 - val_loss: 1.3843 - v

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1006 samples, validate on 607 samples
Epoch 1/20
1006/1006 [==============================] - 1s 561us/sample - loss: 824.6348 - acc: 0.2515 - val_loss: 1.6015 - val_acc: 0.2488
Epoch 2/20
1006/1006 [==============================] - 0s 359us/sample - loss: 1.3831 - acc: 0.2535 - val_loss: 1.3840 - val_acc: 0.2488
Epoch 3/20
1006/1006 [==============================] - 0s 352us/sample - loss: 1.3943 - acc: 0.2555 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 4/20
1006/1006 [==============================] - 0s 326us/sample - loss: 1.4378 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 5/20
1006/1006 [==============================] - 0s 330us/sample - loss: 1.3864 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 6/20
1006/1006 [==============================] - 0s 340us/sample - loss: 1.3863 - acc: 0.2545 - val_loss: 1.3863 - val_acc: 0.2471
Epoch 7/20
1006/1006 [==============================] - 0s 328us/sample - loss: 1.3864 - acc: 0.2545 - val_loss: 1.3863 - v

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
